# Finite differences

operating on a time-dependent quantity 
$$\mathcal{D}(u(\textbf{x},t^n))=\sum_{j\leq 1}\alpha_ju^{n+j}(\textbf{x})$$
$$\mathcal{D}(c(t^n))=\sum_{j\leq 1}\alpha_jc^{n+j}$$

operating on a time-independent quantity
$$\mathcal{D}(u(\textbf{x}))=u(\textbf{x})$$
$$\mathcal{D}(c)=c$$

In [1]:
from lucifex.fdm import FunctionSeries, ExprSeries, FiniteDifference, DT, AB, AM, CN
from lucifex.fem import Constant
from lucifex.mesh import interval_mesh


mesh = interval_mesh(1.0, 10)
order = 3

a = FunctionSeries((mesh, 'P', 1), 'a', order=order)
for t in (0.0, 0.1, 0.2):
    a.update(2 * t)
    a.forward(t) 

print('\n Time derivative:')
dt = 0.001
print(DT(a, dt))
print(DT(a))
dt = Constant(mesh, 0.001, name='dt')
print(DT(a, dt))

print('\n Adams-Bashforth explicit discretizations:')
for n in range(1, order + 1):
    print(AB(n)(a))

print('\n Adams-Moulton implicit discretizations:')
for n in range(1, order + 1):
    print(AM(n)(a))
    print(AM(n)(a, False))

print('\n Custom discretization:')
fd_custom = FiniteDifference({0: 2.0, -1: -1.0}, init=AB(1))
print(fd_custom(a))


 Time derivative:
(v_1 + -1.0 * a⁰) / 0.001
v_1 + -1.0 * a⁰
(v_1 + -1.0 * a⁰) / dt

 Adams-Bashforth explicit discretizations:
a⁰
1.5 * a⁰ + -0.5 * a⁻¹
1.9166666666666667 * a⁰ + -1.3333333333333333 * a⁻¹ + 0.4166666666666667 * a⁻²

 Adams-Moulton implicit discretizations:
v_1
a⁺¹
0.5 * v_1 + 0.5 * a⁰
0.5 * a⁺¹ + 0.5 * a⁰
0.4166666666666667 * v_1 + 0.6666666666666666 * a⁰ + -0.08333333333333333 * a⁻¹
0.4166666666666667 * a⁺¹ + 0.6666666666666666 * a⁰ + -0.08333333333333333 * a⁻¹

 Custom discretization:
2.0 * a⁰ + -1.0 * a⁻¹


In [ ]:
c = Constant(mesh, 0.1)

for n in range(1, order + 1):
    print(AM(n)(c))


## Argument-wise application of finite differences

$$(\mathcal{D}_0 \circ \mathcal{D}_1 \circ \dots)(f(u_0, u_1, \dots)) = f(\mathcal{D}_0(u_0), \mathcal{D}_1(u_1), \dots)$$


In [2]:
b = FunctionSeries((mesh, 'P', 1), 'b', order=order)
for t in (0.0, 0.1, 0.2):
    b.update(-2 * t)
    b.forward(t) 

f = ExprSeries.from_args(a, b, lambda a, b: a * b, name='f')

repr(f)

'ExprSeries(a⁻² * b⁻², a⁻¹ * b⁻¹; Unsolved; Unsolved)'

In [3]:
fd = AB(2) @ CN

print(fd(f))
print(fd(f, trial=b))

(1.5 * a⁰ + -0.5 * a⁻¹) * (0.5 * b⁺¹ + 0.5 * b⁰)
(1.5 * a⁰ + -0.5 * a⁻¹) * (0.5 * v_1 + 0.5 * b⁰)
